In [6]:
import numpy as np,codecs
import os
import math
s='/content/drive/MyDrive/mnist/'
files=os.listdir(s)

def gint(b):
    return int(codecs.encode(b,'hex'),16)

dic={}

for file in files:
    if file.endswith('ubyte'):
        print('Reading',file)
        with open(s+file,'rb') as f:
            data=f.read()
            mnam=gint(data[:4])
            nosam=gint(data[4:8])
            if mnam==2049:
                cat='label'
                parsed=np.frombuffer(data,dtype=np.uint8,offset=8)
                parsed=parsed.reshape(nosam)
                print(parsed)
            elif mnam==2051:
                cat='Image'
                nor=gint(data[8:12])
                noc=gint(data[12:16])
                parsed=np.frombuffer(data,dtype=np.uint8,offset=16)
                parsed=parsed.reshape(nosam,nor,noc)
            if nosam==10000:
                e='test'
            else:
                e='train'
            dic[cat+'_'+e]=parsed
w1=dic['Image_test'].reshape(dic['Image_test'].shape[0],784)
w2=dic['Image_train'].reshape(dic['Image_train'].shape[0],784)
l2 = np.atleast_1d(np.linalg.norm(w1, 2, 1))
l2[l2 == 0] = 1
w1= w1 / np.expand_dims(l2, 1)
l2 = np.atleast_1d(np.linalg.norm(w2, 2, 1))
l2[l2 == 0] = 1
w2=w2/np.expand_dims(l2,1)

In [ ]:
def sigmoid(x):
    return 1/(1+np.exp(-x))
def dersig(x):
    return sigmoid(x)*(1-sigmoid(x))
def tran(x):
    e=int(x)
    q=np.zeros((10,1))
    q[e]=1
    return q
def error(il3,y):
    z=il3-y
    e=np.argmax(y)
    z[e]+=2*(il3[e]-y[e])
    return 0.5*(z.T).dot(z)
def feedforw(x1,sow,y,p,wd,w1d):
    nol=len(sow)
    ld={}
    x=x1.reshape(x1.size,1)
    ld[0]=x
    for i in range(1,nol+1):
        
        #print(sow[i][-1,:].shape)
        il=sigmoid(np.dot((sow[i][0:-1,:].reshape((sow[i].shape[0]-1,sow[i].shape[1]))).T,ld[i-1])+sow[i][-1,:].reshape((sow[i].shape[1],1)))
        il=il.reshape((il.size,1))
        e=np.random.randn(il.size)
        for j in range(len(e)):
            if e[j]<p[i]:
                il[j]=0
        #print(il.shape)
        ld[i]=il
        #print(ld[i].shape)
        
    gd={}
    ed={}
    x=x1.reshape(x1.size,1)
    ed[nol+1]=y-ld[nol]
    v=np.argmax(y)
    ed[nol+1][v]=2*(y[v]-ld[nol][v])
    for i in range(nol,0,-1):
        gd[i]=np.zeros(sow[i].shape)
        if i==nol:
            ed[nol]=ld[nol]*(1-ld[nol])*ed[nol+1]
        else:
            ed[i]=ld[i]*(1-ld[i])*(sow[i+1][0:-1,:].dot(ed[i+1]))
        gd[i][0:-1,:]=ld[i-1].dot(ed[i].T)+wd*sow[i]+w1d*np.sign(sow[i])
        gd[i][-1,:]=ed[i].T+wd+w1d
    
    return gd

def pert(x1,sow,y):
    nol=len(sow)
    ld={}
    x=x1.reshape(x1.size,1)
    ld[0]=x
    for i in range(1,nol+1):
        il=sigmoid(np.dot((sow[i][0:-1,:].reshape((sow[i].shape[0]-1,sow[i].shape[1]))).T,ld[i-1])+sow[i][-1,:].reshape((sow[i].shape[1],1)))
        il=il.reshape((il.size,1))
        #print(il.shape)
        ld[i]=il
        #print(ld[i].shape)
    z=ld[nol]-y
    #print(il3)
    if y[int(np.argmax(ld[nol]))]==1:
        return 0.5*(z.T).dot(z),1
    else:
        return 0.5*(z.T).dot(z),0
def trainz(x1,y,epochs,alpha,nil,batch_size=64,p,wd,w1d):
    np.random.seed(0)
    sow={}
    zi=np.arange(y.size)
    l1=[]
    l2=[]
    l3=[]
    np.random.shuffle(zi)
    for i in range(1,len(nil)):
        sow[i]=np.random.randn(nil[i-1]+1,nil[i])
        #print(nil[i-1]+1,nil[i])
        
    for j in range(epochs):
        x=0
        err=0
        sowe={}
        print('Epoch no.'+str(j+1))
        for i in range(1,len(nil)):
            sowe[i]=np.zeros((nil[i-1]+1,nil[i]))
        nop=0
        err=0
        t=0
        for k in range(math.floor(int(y.size)*0.8)):
            t+=1
            gd=feedforw(x1[int(zi[k])],sow,tran(y[int(zi[k])]),p,wd,w1d)
            for i in range(1,len(nil)):
                sowe[i]+=gd[i]
            if (k!=0 and k%batch_size==0) or k==math.floor(int(y.size))-1:
                #print(t)
                for i in range(1,len(nil)):
                    sow[i]+=alpha*(sowe[i]/t)
                    sowe[i]=sowe[i]/(batch_size/2)
                t=0
        
        for k in range(math.floor(int(y.size)*0.8),math.floor(int(y.size))):
            d,e=pert(x1[int(zi[k])],sow,tran(y[int(zi[k])]))
            x+=e
            err+=float(d)
        print(x)
        print('true positive='+str(x/math.floor(int(y.size)*0.8)))
        print('mean error='+str(err/math.floor(int(y.size)*0.8)))
        l1.append(j)
        l2.append(x/math.floor(int(y.size)*0.8))
        x=0
        err=0
        for k in range(math.floor(int(y.size)*0.8),y.size):
            d,e=pert(x1[int(zi[k])],sow,tran(y[int(zi[k])]))
            x+=e
            err+=float(d)

        
        print(x)
        print('true positive='+str(5*x/int(y.size)))
        print('mean error='+str(5*err/int(y.size)))
        l3.append(5*x/int(y.size))
        if ear_stop(l3):
            break
    return sow,l1,l2,l3
def ear_stop(x):
    if len(x)>7:
        x[len(x)-1]<[len(x)-7]:
            return True
    return False
            
def trainzu(x1,y,epochs,alpha,nil,sow,batch_size=32):
    np.random.seed(0)
    #sow={}
    zi=np.arange(y.size)
    l1=[]
    l2=[]
    l3=[]
    np.random.shuffle(zi)
    #for i in range(1,len(nil)):
        #sow[i]=np.random.randn(nil[i-1]+1,nil[i])
        #print(nil[i-1]+1,nil[i])
        
    for j in range(epochs):
        x=0
        err=0
        sowe={}
        print('Epoch no.'+str(j+1))
        for i in range(1,len(nil)):
            sowe[i]=np.zeros((nil[i-1]+1,nil[i]))
        nop=0
        err=0
        t=0
        for k in range(math.floor(int(y.size)*0.8)):
            t+=1
            gd=feedforw(x1[int(zi[k])],sow,tran(y[int(zi[k])]))
            for i in range(1,len(nil)):
                sowe[i]+=gd[i]
            if (k!=0 and k%batch_size==0) or k==math.floor(int(y.size))-1:
                for i in range(1,len(nil)):
                    sow[i]+=sowe[i]/t
                    sowe[i]=sowe[i]/(batch_size/2)
                t=0
        
        for k in range(math.floor(int(y.size)*0.8),math.floor(int(y.size))):
            d,e=pert(x1[int(zi[k])],sow,tran(y[int(zi[k])]))
            x+=e
            err+=float(d)
        print(x)
        print('true positive='+str(x/math.floor(int(y.size)*0.8)))
        print('mean error='+str(err/math.floor(int(y.size)*0.8)))
        l1.append(j+150)
        l2.append(x/math.floor(int(y.size)*0.8))
        x=0
        err=0
        for k in range(math.floor(int(y.size)*0.8),y.size):
            d,e=pert(x1[int(zi[k])],sow,tran(y[int(zi[k])]))
            x+=e
            err+=float(d)

        
        print(x)
        print('true positive='+str(5*x/int(y.size)))
        print('mean error='+str(5*err/int(y.size)))
        l3.append(5*x/int(y.size))
    return sow,l1,l2,l3